In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle
import pandas as pd

In [7]:
Data =pd.read_csv('sample_regression_data.csv')

In [8]:
Data

,Feature1,Feature2,Feature3,Feature4,Feature5,Target
0,43.708611,32.101582,141.249548,0,3,43.51
1,95.564288,4.206998,461.021163,1,3,126.69
2,75.879455,8.081436,302.100949,1,1,109.04
3,63.879264,44.927709,430.582986,1,2,142.20
4,24.041678,30.321453,228.019840,0,2,79.91
...,...,...,...,...,...,...
195,41.428862,46.537866,289.584656,1,4,75.76
196,75.336011,42.920638,367.023095,0,4,112.14
197,90.739923,21.449701,168.927948,0,1,78.63
198,89.837778,37.543553,176.915608,1,4,78.93


In [9]:
X=Data.drop(['Target'],axis=1)
y=Data['Target']

In [10]:
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=42)
scl=StandardScaler()
X_train =scl.fit_transform(X_train)
X_test=scl.transform(X_test)

In [25]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scl, file)

In [11]:
(X_train.shape[1],)

(5,)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard


In [13]:
new_train = Sequential([Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
                        Dense(32,activation='relu'),
                       Dense(1)])

c:\Users\dussa\OneDrive\Desktop\Python\Krish\.conda\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
new_train.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,497 (9.75 KB)

 Trainable params: 2,497 (9.75 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
import tensorflow
opt =tensorflow.keras.optimizers.Adam(learning_rate=0.2)
new_train.compile(optimizer=opt,loss='mean_absolute_error',metrics=['mae'])

In [16]:
import datetime
log_dir = "regression/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensor=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [17]:
early =EarlyStopping(monitor='val_loss',patience =10,restore_best_weights=True)

In [18]:
trained_model =new_train.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensor,early])

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - loss: 64.5249 - mae: 64.5249 - val_loss: 32.4874 - val_mae: 32.4874
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 22.4516 - mae: 22.4516 - val_loss: 18.7088 - val_mae: 18.7088
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 16.8847 - mae: 16.8847 - val_loss: 13.4182 - val_mae: 13.4182
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 12.3860 - mae: 12.3860 - val_loss: 11.5636 - val_mae: 11.5636
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 10.4485 - mae: 10.4485 - val_loss: 12.6337 - val_mae: 12.6337
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 10.4438 - mae: 10.4438 - val_loss: 15.2163 - val_mae: 15.2163
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 14.2434 - mae: 14.2434 - val_loss: 19.1290 - val_mae: 19.1290
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 17.3574 - mae: 17.3574 - val_loss: 11.9281 - val_mae: 11.9281
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [23]:
new_train.save('regression_model.h5')

In [24]:
%load_ext tensorboard
%tensorboard --logdir regression

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 31464), started 0:03:44 ago. (Use '!kill 31464' to kill it.)